In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Load API key
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)

### RAG

In [2]:
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

with open("sample.txt", "r", encoding="utf-8") as f:
    text_data = f.read()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
)
texts = splitter.split_text(text_data)

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

vectorstore = FAISS.from_texts(texts,embeddings)

#vectorstore = Chroma.from_texts(texts, embedding=embeddings, persist_directory="./chroma_db")
retriever = vectorstore.as_retriever()

print(retriever)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"     # simplest style: stuff retrieved chunks directly into prompt
)

query = "Summarize what LangChain is and its core purpose."
answer = qa_chain.invoke({"query": query})
print("Answer:\n", answer["result"])

tags=['FAISS', 'GoogleGenerativeAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E7FFDA4050> search_kwargs={}
Answer:
 LangChain is an open-source framework created by Harrison Chase that simplifies the development of applications powered by large language models (LLMs). It provides tools and standard interfaces for integrating with various LLMs and external resources, allowing developers to build complex and modular AI-driven solutions like chatbots, document Q&A systems, and AI workflows. Its core functionalities include Chains, Agents, Prompt Management, Memory, and Integrations.


## TYPES OF RETRIEVERS

### ContextualCompressionRetriever

In [3]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectorstore.as_retriever()
)

# Inspect retrieved compressed docs
result = compression_retriever.get_relevant_documents("Explain LangChain in short")

for doc in result:
    print("-",doc.page_content)

# Or use in a QA chain
#from langchain.chains import RetrievalQA
#qa = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)
#qa.invoke({"query": "Explain LangChain in short"})


C:\Users\admin\AppData\Local\Temp\ipykernel_31712\3954767240.py:11: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = compression_retriever.get_relevant_documents("Explain LangChain in short")


- LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It provides a standard interface for integrating with various LLMs and external tools, enabling the creation of complex, modular, and advanced AI-driven solutions.
- LangChain is a framework for building applications with LLMs.LangChain was created by Harrison Chase.LangChain supports RAG, agents, memory, tools, and more.It’s commonly used in chatbots, document Q&A, and AI workflow
- LangChain (Core Framework): 
This is the foundation, providing the core abstractions and interfaces for building LLM applications. It includes functionalities for:
Chains: Combining LLMs with other components (like data sources, tools) into sequences of operations.
Agents: Allowing LLMs to interact with their environment, use tools, and perform multi-step reasoning.
Prompt Management: Tools for creating, managing, and optimizing prompts for LLMs.


### Integration with ConversationalRetrievalChain- Without agent mode

In [4]:
#Integration with ConversationalRetrievalChain- Without agent mode
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

# ✅ ContextualCompressionRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectorstore.as_retriever()
)

# ✅ Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ✅ Chain
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=compression_retriever,
    memory=memory
)

# 🧪 Ask a few questions
print("\n ConversationalRetrievalChain:")
print(rag_chain.invoke({"question": "What is LangChain?"})["answer"])
print(rag_chain.invoke({"question": "Who created it?"})["answer"])



 ConversationalRetrievalChain:


C:\Users\admin\AppData\Local\Temp\ipykernel_31712\975304748.py:16: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It provides the core abstractions and interfaces for building LLM applications, such as chains, agents, and prompt management tools. It supports RAG, agents, memory, tools, and more, and it's commonly used in chatbots, document Q&A, and AI workflows. LangChain was created by Harrison Chase.
LangChain was created by Harrison Chase.


### Integration with Agent (using Tool)

In [6]:
#Integration into Agent (with Tool)

from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool

# 5. Wrap RAG as a StructuredTool
def rag_tool_fn(question: str) -> str:
    return rag_chain.invoke({
        "question": question,
        "chat_history": [memory.chat_memory.messages ]
    })["answer"]

# Structured Tool
rag_tool = StructuredTool.from_function(
    name="RAG_Tool",
    description="Answer LangChain-related questions with context.",
    func=rag_tool_fn
)

# Agent with memory
agent = initialize_agent(
    tools=[rag_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# Ask via agent
print("\n Agent Conversation:")
print(agent.run("What is LangChain?"))
print(agent.run("Who created it?"))



 Agent Conversation:


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
AI: LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It provides the core abstractions and interfaces for building LLM applications, such as chains, agents, and prompt management tools. It supports RAG, agents, memory, tools, and more, and it's commonly used in chatbots, document Q&A, and AI workflows.
```

> Finished chain.
LangChain is an open-source framework designed to simplify the development of applications powered by large language models (LLMs). It provides the core abstractions and interfaces for building LLM applications, such as chains, agents, and prompt management tools. It supports RAG, agents, memory, tools, and more, and it's commonly used in chatbots, document Q&A, and AI workflows.
```


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
AI: LangChain was c

### MultiQueryRetriever

In [7]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import RetrievalQA

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm
)

rag_multi = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=multi_query_retriever,
    return_source_documents = True)

print("RAG pipeline (multiqueryretriever)")
res = rag_multi.invoke({"query":"tell me about langchain creator and its features"})
print("Answer:",res["result"])

#print("\n sources")
#for doc in res["source_documents"]:
#    print("-",doc.page_content[:200])

#docs = multi_query_retriever.get_relevant_documents("Who developed LangChain?")


RAG pipeline (multiqueryretriever)
Answer: LangChain was created by Harrison Chase. It is an open-source framework designed to simplify the development of applications powered by large language models (LLMs).

Key features of LangChain include:

*   **Chains:** Combining LLMs with other components (like data sources, tools) into sequences of operations.
*   **Agents:** Allowing LLMs to interact with their environment, use tools, and perform multi-step reasoning.
*   **Prompt Management:** Tools for creating, managing, and optimizing prompts for LLMs.
*   **Memory:** Storing and retrieving conversational history for stateful interactions.
*   **Integrations:** Connecting with various LLMs, vector stores, document loaders, and other tools.

It supports RAG (Retrieval-Augmented Generation). It is commonly used in chatbots, document Q&A, and AI workflow.


### PARENT DOCUMENT RETRIEVER

In [8]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain.schema.document import Document

# 1. Prepare documents
docs = [Document(page_content="LangChain helps build LLM-powered apps with memory and agents.", metadata={"id": "1"}),
        Document(page_content="Agents in LangChain use tools to answer questions.", metadata={"id": "2"})]

# 2. Setup child splitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)

# 3. Setup vectorstore for children
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

vectorstore = FAISS.from_texts(texts,embeddings)

# 4. Parent retriever
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=InMemoryStore(),  # Stores parent docs
    child_splitter=child_splitter
)

# 5. Add documents
retriever.add_documents(docs)

# 6. Retrieve
results = retriever.get_relevant_documents("What are agents?")
for doc in results:
    print(" Retrieved Doc:", doc.page_content)


 Retrieved Doc: Agents in LangChain use tools to answer questions.
 Retrieved Doc: LangChain helps build LLM-powered apps with memory and agents.


### BM25 RETRIEVER

In [9]:
#!pip install rank_bm25

In [10]:
from langchain.retrievers import BM25Retriever
from langchain.schema.document import Document

# Create simple text docs
docs = [
    Document(page_content="LangChain enables LLM applications."),
    Document(page_content="Vector search is powerful."),
    Document(page_content="BM25 is a classical retrieval method.")
]

# Create BM25 retriever
bm25_retriever = BM25Retriever.from_documents(docs)

# Retrieve
results = bm25_retriever.get_relevant_documents("embedding")
print("RESULT:",results)
for doc in results:
    print(" BM25 Result:", doc.page_content)

RESULT: [Document(metadata={}, page_content='BM25 is a classical retrieval method.'), Document(metadata={}, page_content='Vector search is powerful.'), Document(metadata={}, page_content='LangChain enables LLM applications.')]
 BM25 Result: BM25 is a classical retrieval method.
 BM25 Result: Vector search is powerful.
 BM25 Result: LangChain enables LLM applications.


### Ensemble Retriever

In [11]:
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever

#retriever1 = vectorstore.as_retriever(search_kwargs={"k": 3})
#retriever2 = multi_query_retriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectorstore.as_retriever()
)

ensemble = EnsembleRetriever(
    retrievers=[multi_query_retriever, compression_retriever],
    weights=[0.5, 0.5]
)
qa = RetrievalQA.from_chain_type(llm=llm, retriever=ensemble)
qa.invoke({"query": "What is LangChain used for?"})

{'query': 'What is LangChain used for?',
 'result': 'LangChain is used for building applications powered by large language models (LLMs). It provides tools to create chatbots, document Q&A systems, and AI workflows. It supports RAG (Retrieval-Augmented Generation), agents, memory, and tools. It helps combine LLMs with other components like data sources into sequences of operations.'}

In [12]:
from langchain.retrievers import EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_fn = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

vectorstore = FAISS.from_documents(docs, embedding_fn)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

ensemble = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_retriever],
    weights=[0.3, 0.7]
)

results = ensemble.get_relevant_documents("vector")
for doc in results:
    print("Ensemble Result:", doc.page_content)


Ensemble Result: Vector search is powerful.
Ensemble Result: LangChain enables LLM applications.
Ensemble Result: BM25 is a classical retrieval method.


### TimeWeightVectorStore Retriever

#### There is no built-in TimeWeightedVectorStoreRetriever in LangChain 0.3.25. - Use custom time decay / use Enseble Retriever / Switch tolanggraph memory

In [20]:
import time
from datetime import datetime, timedelta
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.schema import Document
from langchain_community.vectorstores import FAISS

# 1. Embeddings
embedding = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# 2. Store docs with timestamps
docs = [
    Document(
        page_content="LangChain helps build LLM apps.",
        metadata={"time": datetime.now() - timedelta(minutes=30)}  # 30 mins old
    ),
    Document(
        page_content="Vector search improves relevance.",
        metadata={"time": datetime.now() - timedelta(minutes=5)}  # 5 mins old = fresher
    )
]

# 3. Build vectorstore
vectorstore = FAISS.from_documents(docs, embedding)

# 4. Normal retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# 5. Apply TIME WEIGHT after vector search
def time_weighted_results(query):
    results = retriever.get_relevant_documents(query)
    now = datetime.now()

    scored = []
    for d in results:
        age_minutes = (now - d.metadata["time"]).total_seconds() / 60
        time_score = 1 / (1 + age_minutes)   # Smaller age → bigger score
        scored.append((time_score, d))

    scored.sort(reverse=True, key=lambda x: x[0])
    return [d for (_, d) in scored]


# TEST
results = time_weighted_results("What is LangChain?")
for r in results:
    print("Doc:", r.page_content)
    print("Timestamp:", r.metadata["time"])


Doc: Vector search improves relevance.
Timestamp: 2025-11-14 13:08:40.332532
Doc: LangChain helps build LLM apps.
Timestamp: 2025-11-14 12:43:40.332478
